In [259]:
import numpy as np
import math
import matplotlib.pyplot as plt

In [260]:
def splitData(data, ratio):
    np.random.shuffle(data)
    train_size = int(len(data) * ratio)
    train_set = data[:train_size]
    test_set = data[train_size:]
    return train_set, test_set

In [261]:
def timeToMinutes(time):
    splitted = time.split(":")
    if int(splitted[0]) > 23 or int(splitted[1]) > 59:
        raise ValueError
    return int(splitted[0]) * 60 + int(splitted[1])

In [262]:
def processTime(time):
    minutes = timeToMinutes(time)
    return  math.sin(minutes / (24 * 60)*math.pi)

In [263]:
def loadData(codesToIndexes):
    data = []
    for i in range(1, 70):
        filePrefix = 'Diabetes-Data/data-'
        if i < 10:
            filePrefix += '0'
        fileHandle = open(filePrefix+str(i), 'r')
        for line in fileHandle:
            line = line.strip()
            if line:
                try:
                    parts = line.split('\t')
                    time = processTime(parts[1])
                    code = int(parts[2])
                    if code not in codesToIndexes.keys():
                        raise ValueError
                    number = int(parts[3])
                    data.append([time, code, number])
                except ValueError:
                    pass
                except IndexError:
                    pass
        
        fileHandle.close()
    return data


In [264]:
def encodeOutput(Y,codesToIndexes):
    codesToIndexes = {33:0, 34:1, 35:2, 48:3, 57:4, 58:5, 59:6, 60:7, 61:8, 62:9, 63:10, 64:11, 65:12, 66:13, 67:14, 68:15, 69:16, 70:17, 71:18, 72:19}
    Y_encoded = np.zeros((Y.shape[0], 20))
    for i, code in enumerate(Y):
        index = codesToIndexes[code]
        Y_encoded[i, index] = 1
    return Y_encoded

In [265]:
codesToIndexes = {33:0, 34:1, 35:2, 48:3, 57:4, 58:5, 59:6, 60:7, 61:8, 62:9, 63:10, 64:11, 65:12, 66:13, 67:14, 68:15, 69:16, 70:17, 71:18, 72:19}
data = np.array(loadData(codesToIndexes))
data, test = splitData(data, 0.8)
X = data[:, [0, 2]]
X_test = test[:, [0, 2]]
Y = data[:, 1]
Y_test = test[:, 1]
Y_encoded = encodeOutput(Y,codesToIndexes)
Y_test_encoded = encodeOutput(Y_test,codesToIndexes)

In [266]:
def initializeParameters(nX, nH, nY):
    W1 = np.random.randn(nX, nH)
    b1 = np.zeros((1,nH))
    b1 = np.random.randn(1,nH)
    W2 = np.random.randn(nH,nY)
    b2 = np.zeros((1,nY))
    b2 = np.random.randn(1,nY)
    parameters = {"W1": W1, "b1": b1, "W2": W2, "b2": b2}
    return parameters

### Architektura sieci neuronowej

- **Warstwa wejściowa:** N neurony (wejścia)
- **Warstwy ukryte:** L warstw, każda z dowolną liczbą neuronów
- **Warstwa wyjściowa:** K neuronów (klasyfikacja na K kategorii)

### Funkcje aktywacji

- **Warstwy ukryte:** Sigmoid
  - $$\sigma(z) = \frac{1}{1 + e^{-z}}$$
- **Warstwa wyjściowa:** Softmax
  - $$\text{Softmax}(z_i) = \frac{e^{z_i}}{\sum_{j=1}^{K} e^{z_j}}$$

### Propagacja w przód

Dla każdej warstwy l (od 1 do L+1, gdzie L+1 to warstwa wyjściowa):

1. **Sygnał wejściowy:** $$Z^{[l]} = W^{[l]} A^{[l-1]} + b^{[l]}$$
2. **Aktywacja:** 
   - Dla warstw ukrytych: $$A^{[l]} = \sigma(Z^{[l]})$$
   - Dla warstwy wyjściowej: $$A^{[L+1]} = \text{Softmax}(Z^{[L+1]})$$

### Wsteczna propagacja błędu

1. **Błąd na wyjściu (warstwa L+1):**
   - $$\delta^{[L+1]} = A^{[L+1]} - Y$$

Dla każdej warstwy l od L do 1:

2. **Błąd dla warstwy l:**
   - $$\delta^{[l]} = (W^{[l+1]T} \delta^{[l+1]}) \odot \sigma'(Z^{[l]})$$
3. **Gradient dla wag i biasów:**
   - $$\nabla W^{[l]} = \delta^{[l]} A^{[l-1]T}$$
   - $$\nabla b^{[l]} = \sum(\delta^{[l]}, \text{axis} = 0)$$

### Aktualizacja wag

- Wagi i biasy są aktualizowane za pomocą metody spadku gradientu:
  - $$W^{[l]} = W^{[l]} - \alpha \nabla W^{[l]}$$
  - $$b^{[l]} = b^{[l]} - \alpha \nabla b^{[l]}$$


In [267]:
def costFunction(Y, Y_hat):
    return - np.sum(Y * np.log(Y_hat))

In [268]:
def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

In [269]:
def softmax(Z):
    return np.exp(Z) / np.sum(np.exp(Z), axis=1, keepdims=True)

In [270]:
def propagate(X, Y, parameters):
    Z1 = np.dot(X, parameters["W1"]) + parameters["b1"]
    A1 = sigmoid(Z1)
    Z2 = np.dot(A1, parameters["W2"]) + parameters["b2"]
    A2 = softmax(Z2)
    cost = costFunction(Y, A2)
    # pochodne funkcji straty:
    dA2 = A2 - Y
    dW2 = np.dot(A1.T,dA2) 
    db2 = np.sum(dA2, axis=0,keepdims=True)
    dA1 = np.dot(dA2, parameters["W2"].T)
    dZ1 = dA1 * (A1 * (1 - A1))
    dW1 = np.dot(X.T, dZ1)
    db1 = np.sum(dZ1, axis=0)
    gradients = {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}
    return gradients, cost

In [271]:
def updateParameters(W1, b1, W2, b2, gradients, learningRate):
    W1 = W1 - learningRate * gradients["dW1"]
    b1 = b1 - learningRate * gradients["db1"]
    W2 = W2 - learningRate * gradients["dW2"]
    b2 = b2 - learningRate * gradients["db2"]
    return W1, b1, W2, b2

In [272]:
def optimize(X, Y, parameters, learningRate, numberOfIterations, printCost=False):
    costs = []
    for i in range(numberOfIterations):
        gradients, cost = propagate(X, Y, parameters)
        parameters = updateParameters(parameters, gradients, learningRate)
        costs.append(cost)
        if i % 100 == 0 and printCost:
            print("Cost after iteration {}: {}".format(i, cost))
    return parameters, costs

In [273]:
def predict(X, parameters):
    Z1 = np.dot(X, parameters["W1"]) + parameters["b1"]
    A1 = sigmoid(Z1)
    Z2 = np.dot(A1, parameters["W2"]) + parameters["b2"]
    A2 = softmax(Z2)
    return np.argmax(A2, axis=1)

In [274]:
parameters = initializeParameters(2, 15, 20)
params, costs = optimize(X,Y_encoded, parameters["W1"], parameters["b1"], parameters["W2"], parameters["b2"], 0.00001, 5000, True)

/tmp/ipykernel_10313/70635953.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-Z))


Cost after iteration 0: 116117.91811552043
Cost after iteration 100: 44520.01935853351
Cost after iteration 200: 40562.47933919337
Cost after iteration 300: 40899.934045497095
Cost after iteration 400: 38917.756653436714
Cost after iteration 500: 39420.437505948306
Cost after iteration 600: 38259.836810596295
Cost after iteration 700: 33248.70429841142
Cost after iteration 800: 32549.342267071563
Cost after iteration 900: 31527.49687447481
Cost after iteration 1000: 31643.587158702914
Cost after iteration 1100: 31040.126263063536
Cost after iteration 1200: 30284.661600678264
Cost after iteration 1300: 31448.474174188
Cost after iteration 1400: 29903.25457135787
Cost after iteration 1500: 29926.563232413995
Cost after iteration 1600: 29508.331885950796
Cost after iteration 1700: 29423.715652237137
Cost after iteration 1800: 29326.70786634379
Cost after iteration 1900: 30859.05621527065
Cost after iteration 2000: 29392.540338271017
Cost after iteration 2100: 30209.516087794265
Cost after

In [275]:
a = predict(X, params)
i=18
print(a[i])
sum =0
timesGuessedA = {}
timesGuessedY = {}
trafione = {}
for i in range(a.shape[0]):
    timesGuessedA[a[i]] = timesGuessedA.get(a[i], 0) + 1    
    timesGuessedY[np.argmax(Y_encoded[i])] = timesGuessedY.get(np.argmax(Y_encoded[i]), 0) + 1
    if a[i] == np.argmax(Y_encoded[i]):
        trafione[a[i]] = trafione.get(a[i], 0) + 1
        sum += 1

print(sum,a.shape[0])
print(timesGuessedA)
print(timesGuessedY)
print(trafione)
print(sum/a.shape[0]*100)

/tmp/ipykernel_10313/70635953.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-Z))


5
11472 22988
{5: 10652, 0: 9016, 14: 1019, 1: 2149, 7: 152}
{7: 2211, 1: 2934, 0: 7483, 5: 2806, 2: 845, 9: 2485, 15: 31, 17: 109, 3: 1496, 11: 705, 4: 774, 10: 186, 12: 255, 14: 282, 13: 122, 18: 79, 8: 48, 19: 72, 16: 53, 6: 12}
{0: 7067, 5: 2785, 1: 1302, 14: 282, 7: 36}
49.90429789455368


In [276]:
a = predict(X_test, params)
i=18
print(a[i])
sum =0
timesGuessedA = {}
timesGuessedY = {}
trafione = {}
for i in range(a.shape[0]):
    timesGuessedA[a[i]] = timesGuessedA.get(a[i], 0) + 1    
    timesGuessedY[np.argmax(Y_test_encoded[i])] = timesGuessedY.get(np.argmax(Y_test_encoded[i]), 0) + 1
    if a[i] == np.argmax(Y_test_encoded[i]):
        trafione[a[i]] = trafione.get(a[i], 0) + 1
        sum += 1

print(sum,a.shape[0])
print(timesGuessedA)
print(timesGuessedY)
print(trafione)
print(sum/a.shape[0]*100)

5
2819 5747
{1: 561, 0: 2231, 5: 2661, 7: 50, 14: 244}
{1: 746, 0: 1863, 9: 633, 5: 673, 3: 363, 11: 199, 7: 547, 2: 208, 4: 215, 10: 33, 19: 22, 14: 44, 12: 76, 18: 19, 13: 32, 17: 30, 16: 15, 8: 18, 6: 8, 15: 3}
{1: 334, 0: 1761, 5: 668, 7: 12, 14: 44}
49.05167913694101


/tmp/ipykernel_10313/70635953.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-Z))


In [277]:
# plt.plot(costs[0:100])